In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install torch torchvision torchmetrics
!pip install torch-fidelity

In [ ]:
# =========================
# 압축 파일은 celeba 폴더에
# =========================

img_zip_file = 'dcgan_front_50_epoch.zip'

In [ ]:
# ==========
# 압축 해제
# ==========

import os
import shutil

source_path = f'/content/drive/MyDrive/celeba/{img_zip_file}'
copy_path = '/content/example.zip'
extract_path = '/content/extracted'

folder_name = os.path.splitext(img_zip_file)[0]

shutil.copy(source_path, copy_path)

if not os.path.exists(extract_path):
    os.makedirs(extract_path)

!unzip -q {copy_path} -d {extract_path}

inner_dir = os.path.join(extract_path, folder_name)

if os.path.isdir(inner_dir):
    for filename in os.listdir(inner_dir):
        src_path = os.path.join(inner_dir, filename)
        dst_path = os.path.join(extract_path, filename)
        shutil.move(src_path, dst_path)
    os.rmdir(inner_dir)

In [ ]:
# ================
# Inception Score
# ================

import torch
from torchmetrics.image.inception import InceptionScore
from torchvision.transforms import ToTensor, Resize
from PIL import Image
import os

# IS 객체 초기화
inception = InceptionScore(normalize=True)

generated_images_path = "/content/extracted"  # 본인 환경에 맞게 설정
image_files = [f for f in os.listdir(generated_images_path) if f.endswith(".png") or f.endswith(".jpg")]

# 배치 크기 설정
batch_size = 100

# 이미지 전처리를 위한 transform
resize_transform = Resize((299, 299))

for i in range(0, len(image_files), batch_size):
    batch_files = image_files[i:i + batch_size]
    batch_tensors = []

    for filename in batch_files:
        img_path = os.path.join(generated_images_path, filename)
        img = Image.open(img_path).convert("RGB")
        img = resize_transform(img)
        img_tensor = ToTensor()(img).unsqueeze(0)
        batch_tensors.append(img_tensor)

    # 현재 batch 이미지들을 하나의 텐서로 합친 후 InceptionScore에 업데이트
    batch_tensors = torch.cat(batch_tensors, dim=0)  # shape: [batch_size, 3, 299, 299]
    inception.update(batch_tensors)

# 모든 배치 업데이트가 끝난 후 최종적으로 점수 계산
score, std = inception.compute()

print("\n")
print(img_zip_file)
print(f"Inception Score: {score:.4f} ± {std:.4f}")